In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
files = glob('forums/*.json')
len(files)

9

In [3]:
forums = []
for f in files:
    with open(f) as fopen:
        forums.extend(json.load(fopen))
        
forums = sorted(list(set(forums)))
len(forums)

3279

In [8]:
forums[1]

'/threads/%E2%9C%88-trip-reviews-airlines-aircaft-types-youve-flown-with-opinions-discussions.1907524'

In [9]:
url = 'https://www.skyscrapercity.com' + forums[1]
r = requests.get(url, timeout = 5.0)
soup = BeautifulSoup(r.content, "lxml")

In [10]:
url

'https://www.skyscrapercity.com/threads/%E2%9C%88-trip-reviews-airlines-aircaft-types-youve-flown-with-opinions-discussions.1907524'

In [14]:
li = soup.find_all('li', {'class': 'pageNav-page'})
max_page = []
for a_ in li:
    try:
        max_page.append(int(a_.text))
    except:
        pass
max_page

[1, 2, 3, 6, 1, 2, 3, 6]

In [18]:
td = soup.find_all('div', {'class': 'bbWrapper'})
td = [td_.text.strip() for td_ in td]
td = [td_ for td_ in td if len(td_)]

In [20]:
def get_url(url):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    li = soup.find_all('li', {'class': 'pageNav-page'})
    max_page = []
    for a_ in li:
        try:
            max_page.append(int(a_.text))
        except:
            pass

    try:
        max_page = max(max_page)
    except:
        max_page = -1
    td = soup.find_all('div', {'class': 'bbWrapper'})
    td = [td_.text.strip() for td_ in td]
    td = [td_ for td_ in td if len(td_)]
    return url, td, max_page

In [21]:
get_url('https://www.skyscrapercity.com' + forums[1])

('https://www.skyscrapercity.com/threads/%E2%9C%88-trip-reviews-airlines-aircaft-types-youve-flown-with-opinions-discussions.1907524',
 ["Not really sure if this is the right place to start the thread or there is a similar thread already. If it isn't, mods please advice and move it to the correct location or remove completely.\n\nOkay, so I thought we should have a thread to discuss about other carriers and their services (since Malaysian airlines have their dedicated threads here already), the aircraft typing of those airlines and your thoughts, and maybe your trip reports (it'd be nice to have them in one place instead of having them in different threads).\n\nI guess I will start with my first trip report video here:\n\nhttps://www.youtube.com/watch?v=c3xCW75mZUk",
  '^^\n\nVery nice trip review - keep it coming lor !!!',
  'I\'ve posted part of this in the Rayani Air thread but this is the report of the full trip.\n\nLink to trip report\n\n\n\nRttlnSnK said:\n\n\n\nThis is my second

In [22]:
!mkdir threads

In [23]:
max_worker = 20

for i in tqdm(range(0, len(forums), max_worker)):
    
    filename = os.path.join('threads', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
        
    topics = [('https://www.skyscrapercity.com' + topic,) for topic in forums[i: i + max_worker]]
    
    data = []    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_url, url[0]): url for url in topics}

        for future in as_completed(futures):
            data.append(future.result())
                
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)
        
    try:
        del data
        del futures
    except:
        pass

100%|███████████████████████████████████████████████████████████████████████████████████████████| 164/164 [04:59<00:00,  1.82s/it]
